### LSH Prototype 1

In [1]:
import os
import pandas as pd
import numpy as np
from dask.distributed import Client
import dask.array as da
import dask.dataframe as dd
import xarray as xr
%run {os.environ['NB_DIR']}/nb.py
%run $BENCHMARK_METHOD_DIR/common.py
sample_rate = .05
ds_name = DATASET_HM
ds_config = DATASET_CONFIG[ds_name]
ds_path = dataset_path(ds_name, sr=sample_rate)

In [2]:
client = Client(processes=True, threads_per_worker=1, n_workers=2, memory_limit='50GB')
client

Client Scheduler: tcp://127.0.0.1:38449 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 2 Cores: 2 Memory: 100.00 GB


In [3]:
xa = xr.DataArray(
    data=da.from_zarr(ds_path + '.zarr'), 
    dims=['variant', 'sample']
)
xa = xa.assign_coords(variant=da.arange(xa['variant'].shape[0]))
xa = xa.assign_coords(sample=da.arange(xa['sample'].shape[0]))
xa

<xarray.DataArray 'from-zarr-9a0b988399fe1becc131e03d28fbeb35' (variant: 72732, sample: 165)>
dask.array<from-zarr, shape=(72732, 165), dtype=uint8, chunksize=(72732, 165), chunktype=numpy.ndarray>
Coordinates:
  * variant  (variant) int64 0 1 2 3 4 5 ... 72726 72727 72728 72729 72730 72731
  * sample   (sample) int64 0 1 2 3 4 5 6 7 ... 157 158 159 160 161 162 163 164

In [4]:
xa.shape

(72732, 165)

In [5]:
def projection_matrix(dim, n, p, chunks='auto'):
    # TODO: Center or unit normalize?
    return xr.DataArray(
        data=da.random.normal(size=(n, p), chunks=chunks), 
        dims=(dim, 'projection')
    )

In [6]:
X = xa
m = 250
h = 50
g = 3

R = projection_matrix(X.dims[1], X.shape[1], m)
P = X @ R

In [7]:
P

<xarray.DataArray (variant: 72732, projection: 250)>
dask.array<sum-aggregate, shape=(72732, 250), dtype=float64, chunksize=(72732, 250), chunktype=numpy.ndarray>
Coordinates:
  * variant  (variant) int64 0 1 2 3 4 5 ... 72726 72727 72728 72729 72730 72731
Dimensions without coordinates: projection

In [8]:
def hash_matrix(P, h, g):
    H = P[:, :(h*g)]
    return H.assign_coords(hash_group=('projection', H['projection']//h))

In [9]:
H = hash_matrix(P, h, g)
H

<xarray.DataArray (variant: 72732, projection: 150)>
dask.array<getitem, shape=(72732, 150), dtype=float64, chunksize=(72732, 150), chunktype=numpy.ndarray>
Coordinates:
  * variant     (variant) int64 0 1 2 3 4 5 ... 72727 72728 72729 72730 72731
    hash_group  (projection) int64 0 0 0 0 0 0 0 0 0 0 0 ... 2 2 2 2 2 2 2 2 2 2
Dimensions without coordinates: projection

In [11]:
def hash_group(x):
    xs = np.apply_along_axis(lambda v: str(int(''.join(v), 2)), 1, x)
    return xr.DataArray(xs, dims=['variant'])

# Convert float projections to composite hash values
HG = xr.where(H > 0, '1', '0')
HG = HG.groupby('hash_group').apply(hash_group)
HG

<xarray.DataArray (variant: 72732, hash_group: 3)>
array([['332262403273391', '369890042548962', '588710991741146'],
       ['402631147434671', '369615163592930', '587577123525018'],
       ['332262369702575', '370164584911586', '587577103601882'],
       ...,
       ['345456274633391', '352297789395682', '865650557440474'],
       ['332261061620399', '370439529926882', '588126879858906'],
       ['332262403273391', '369887895065314', '588710991745242']],
      dtype='<U15')
Coordinates:
  * hash_group  (hash_group) int64 0 1 2
Dimensions without coordinates: variant

In [18]:
HG.to_dataset(name='hash_value').to_dask_dataframe().head(3)

,hash_group,variant,hash_value
0,0,0,332262403273391
1,0,1,402631147434671
2,0,2,332262369702575


### Try doing self-join w/ xarray

In [12]:
HI = HG.stack(i=['variant', 'hash_group']).reset_index('i')
HI

<xarray.DataArray (i: 218196)>
array(['332262403273391', '369890042548962', '588710991741146', ...,
       '332262403273391', '369887895065314', '588710991745242'],
      dtype='<U15')
Coordinates:
    variant     (i) int64 0 0 0 1 1 1 2 ... 72730 72730 72730 72731 72731 72731
    hash_group  (i) int64 0 1 2 0 1 2 0 1 2 0 1 2 0 ... 0 1 2 0 1 2 0 1 2 0 1 2
Dimensions without coordinates: i

In [80]:
HI = HG.stack(i=['variant', 'hash_group']).reset_index('i')
HV = HI.variant
HV['hv'] = HI
HV = HV.rename({'hash_group': 'hg'})
HV = HV.set_index(i=['hg', 'hv'])

#HI = HI.assign_coords(hg=('i', HI))
#HI['hg'] = HI
#HI = HI.reset_index('i', drop=True)
HV

<xarray.DataArray 'variant' (i: 218196)>
array([    0,     0,     0, ..., 72731, 72731, 72731])
Coordinates:
    variant  (i) int64 0 0 0 1 1 1 2 ... 72730 72730 72730 72731 72731 72731
  * i        (i) MultiIndex
  - hg       (i) int64 0 1 2 0 1 2 0 1 2 0 1 2 0 1 ... 2 0 1 2 0 1 2 0 1 2 0 1 2
  - hv       (i) object '70853237133192' '304379673645668' ... '258141934308455'

In [82]:
HVA = xr.merge(
    [HV.rename({'variant': 'variant_x'}), HV.rename({'variant': 'variant_y'})],
    join='inner'
)

In [83]:
HVA

<xarray.Dataset>
Dimensions:    (i: 218196)
Coordinates:
    variant_x  (i) int64 0 0 0 1 1 1 2 ... 72730 72730 72730 72731 72731 72731
  * i          (i) MultiIndex
  - hg         (i) int64 0 1 2 0 1 2 0 1 2 0 1 2 0 ... 2 0 1 2 0 1 2 0 1 2 0 1 2
  - hv         (i) object '70853237133192' ... '258141934308455'
    variant_y  (i) int64 0 0 0 1 1 1 2 ... 72730 72730 72730 72731 72731 72731
Data variables:
    variant    (i) int64 0 0 0 1 1 1 2 ... 72730 72730 72730 72731 72731 72731

In [88]:
# Zero results here indicate that it's an alignment and not a join (like pandas concat rather than pd.merge)
HVAF = HVA.where(HVA.variant_x != HVA.variant_y, drop=True)
HVAF = HVAF.reset_index('i')
HVAF

<xarray.Dataset>
Dimensions:    (i: 0)
Coordinates:
    variant_x  (i) int64 
    variant_y  (i) int64 
    hg         (i) int64 
    hv         (i) object 
Dimensions without coordinates: i
Data variables:
    variant    (i) float64

### Self-join w/ Dask

In [11]:
df_hg = xr.Dataset({'HG':HG}).to_dask_dataframe(dim_order=('variant', 'hash_group'))
df_hg.head(6)

,variant,hash_group,vid,HG
0,0,0,0,427038954008940
1,0,1,0,931935426143761
2,0,2,0,715685111648468
3,1,0,1,436384811233660
4,1,1,1,931868921259601
5,1,2,1,143939052622036


In [12]:
df = df_hg.compute()

In [92]:
df.groupby(['hash_group', 'HG']).size().max()

7626

In [93]:
df_hgi = df_hg.assign(hgi_hgv=lambda df: df['hash_group'].astype(str) + ':' + df['HG']).set_index(['hgi_hgv'])
df_hgi = df_hgi.repartition(npartitions=16)
df_hgi

,variant,hash_group,HG
npartitions=16,,,
,int64,int64,object
,...,...,...
...,...,...,...
,...,...,...
,...,...,...


In [94]:
df_hgi.head(6)

,variant,hash_group,HG
hgi_hgv,,,
0:10030983028325,27414,0,10030983028325
0:10031015240528,49475,0,10031015240528
0:10032389630073,59236,0,10032389630073
0:1031035935627,26733,0,1031035935627
0:1032035225482,30768,0,1032035225482
0:1032638156685,38459,0,1032638156685


In [95]:
# Note that even though the divisions are not known yet, there appears to be no difference
# if you write the data out and read it back in via:
# df_hgi.to_parquet('/tmp/df_hgi')
# df_hgi = dd.read_parquet('/tmp/df_hgi', infer_divisions=True)
# df_hgi.divisions # divisions are now known
# and simply assuming that dask will compute them prior to the join.
# This is important b/c of: https://docs.dask.org/en/latest/dataframe-design.html#partitions 
df_hgi.divisions[:8]

(None, None, None, None, None, None, None, None)

In [96]:
dfa = dd.merge(
    df_hgi.rename(columns={'variant': 'variant_x'}),
    df_hgi.rename(columns={'variant': 'variant_y'}),
    left_index=True,
    right_index=True
)
dfa = dfa[dfa['variant_x'] != dfa['variant_y']]
dfa = dfa[['variant_x', 'variant_y']].drop_duplicates()
dfa

,variant_x,variant_y
npartitions=1,,
,int64,int64
,...,...


In [97]:
%%time
dfa.to_parquet('/tmp/dfa.parquet')

In [9]:
dfa = dd.read_parquet('/tmp/dfa.parquet').rename(columns={'variant_x': 'vid_x', 'variant_y': 'vid_y'})

In [10]:
dfa

,vid_x,vid_y
npartitions=1,,
0:10589674771444,int64,int64
2:276971070917703,...,...


In [11]:
# Standardized P means dot products will equal pearson correlation
PC = (P - P.mean(axis=1)) / P.std(axis=1)
PC

<xarray.DataArray (variant: 72732, projection: 250)>
dask.array<truediv, shape=(72732, 250), dtype=float64, chunksize=(72732, 250), chunktype=numpy.ndarray>
Coordinates:
    vid      (variant) int64 dask.array<chunksize=(72732,), meta=np.ndarray>
Dimensions without coordinates: variant, projection

In [12]:
def get_dfp(prefix):
    dfp = dd.from_dask_array(PC.data)
    dfp = dfp.rename(columns=lambda c: prefix + str(c))
    dfp['vid'] = PC.vid.data
    return dfp

In [13]:
get_dfp('x').set_index('vid')

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99,x100,x101,x102,x103,x104,x105,x106,x107,x108,x109,x110,x111,x112,x113,x114,x115,x116,x117,x118,x119,x120,x121,x122,x123,x124,x125,x126,x127,x128,x129,x130,x131,x132,x133,x134,x135,x136,x137,x138,x139,x140,x141,x142,x143,x144,x145,x146,x147,x148,x149,x150,x151,x152,x153,x154,x155,x156,x157,x158,x159,x160,x161,x162,x163,x164,x165,x166,x167,x168,x169,x170,x171,x172,x173,x174,x175,x176,x177,x178,x179,x180,x181,x182,x183,x184,x185,x186,x187,x188,x189,x190,x191,x192,x193,x194,x195,x196,x197,x198,x199,x200,x201,x202,x203,x204,x205,x206,x207,x208,x209,x210,x211,x212,x213,x214,x215,x216,x217,x218,x219,x220,x221,x222,x223,x224,x225,x226,x227,x228,x229,x230,x231,x232,x233,x234,x235,x236,x237,x238,x239,x240,x241,x242,x243,x244,x245,x246,x247,x248,x249
npartitions=1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
72731,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [14]:
dfap = dd.merge(
    dfa.set_index('vid_x').repartition(npartitions=32), 
    get_dfp('x').set_index('vid').repartition(npartitions=32), 
    left_index=True, right_index=True,
    how='inner'
).reset_index()
dfap = dd.merge(
    dfap.set_index('vid_y').repartition(npartitions=32), 
    get_dfp('y').set_index('vid').repartition(npartitions=32), 
    left_index=True, right_index=True,
    how='inner'
).reset_index()
dfap

,vid_y,vid_x,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99,x100,x101,x102,x103,x104,x105,x106,x107,x108,x109,x110,x111,x112,x113,x114,x115,x116,x117,x118,x119,x120,x121,x122,x123,x124,x125,x126,x127,x128,x129,x130,x131,x132,x133,x134,x135,x136,x137,x138,x139,x140,x141,x142,x143,x144,x145,x146,x147,x148,x149,x150,x151,x152,x153,x154,x155,x156,x157,x158,x159,x160,x161,x162,x163,x164,x165,x166,x167,x168,x169,x170,x171,x172,x173,x174,x175,x176,x177,x178,x179,x180,x181,x182,x183,x184,x185,x186,x187,x188,x189,x190,x191,x192,x193,x194,x195,x196,x197,x198,x199,x200,x201,x202,x203,x204,x205,x206,x207,x208,x209,x210,x211,x212,x213,x214,x215,x216,x217,x218,x219,x220,x221,x222,x223,x224,x225,x226,x227,x228,x229,x230,x231,x232,x233,x234,x235,x236,x237,x238,x239,x240,x241,x242,x243,x244,x245,x246,x247,x248,x249,y0,y1,y2,y3,y4,y5,y6,y7,y8,y9,y10,y11,y12,y13,y14,y15,y16,y17,y18,y19,y20,y21,y22,y23,y24,y25,y26,y27,y28,y29,y30,y31,y32,y33,y34,y35,y36,y37,y38,y39,y40,y41,y42,y43,y44,y45,y46,y47,y48,y49,y50,y51,y52,y53,y54,y55,y56,y57,y58,y59,y60,y61,y62,y63,y64,y65,y66,y67,y68,y69,y70,y71,y72,y73,y74,y75,y76,y77,y78,y79,y80,y81,y82,y83,y84,y85,y86,y87,y88,y89,y90,y91,y92,y93,y94,y95,y96,y97,y98,y99,y100,y101,y102,y103,y104,y105,y106,y107,y108,y109,y110,y111,y112,y113,y114,y115,y116,y117,y118,y119,y120,y121,y122,y123,y124,y125,y126,y127,y128,y129,y130,y131,y132,y133,y134,y135,y136,y137,y138,y139,y140,y141,y142,y143,y144,y145,y146,y147,y148,y149,y150,y151,y152,y153,y154,y155,y156,y157,y158,y159,y160,y161,y162,y163,y164,y165,y166,y167,y168,y169,y170,y171,y172,y173,y174,y175,y176,y177,y178,y179,y180,y181,y182,y183,y184,y185,y186,y187,y188,y189,y190,y191,y192,y193,y194,y195,y196,y197,y198,y199,y200,y201,y202,y203,y204,y205,y206,y207,y208,y209,y210,y211,y212,y213,y214,y215,y216,y217,y218,y219,y220,y221,y222,y223,y224,y225,y226,y227,y228,y229,y230,y231,y232,y233,y234,y235,y236,y237,y238,y239,y240,y241,y242,y243,y244,y245,y246,y247,y248,y249
npartitions=63,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,f

In [15]:
def reduce_corr(df, prefixes):
    df1 = df[[c for c in df if c.startswith(prefixes[0])]]
    df2 = df[[c for c in df if c.startswith(prefixes[1])]]
    df['corr'] = (df1 * df2).sum(axis=1)
    return df.drop([c for c in df if c.startswith(prefixes[0]) or c.startswith(prefixes[1])], axis='columns')

In [16]:
dfapc = reduce_corr(dfap, ['x', 'y'])
dfapc

,vid_y,vid_x,corr
npartitions=63,,,
,int64,int64,float64
,...,...,...
...,...,...,...
,...,...,...
,...,...,...


In [17]:
%%time
dfapc.to_parquet('/tmp/dfapc.parquet')

distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker


KeyboardInterrupt: 

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker


In [244]:
s = dfapc['corr']

In [252]:
hist_freq, hist_bins = da.histogram(dfapc['corr'], bins=256, range=[-2, 2])
hist_freq = hist_freq.compute()

distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker


KeyboardInterrupt: 

In [ ]:
hist_freq

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
